# Ratings datasets import and cleaning

The IMdB movies dataset imported from IMdB website is huge (882.4 Mo) and therefore long to process. As it contains many unuseful features and many unuseful movies (i.e. not present in the CMU pre-processed dataset), we will reduce its size and combine it to the IMdB ratings dataset (23.7 Mo) and then marge the obtained dataset to the pre-processed CMU dataset. In the end, we will get a pre-processed CMU-IMdB dataset that is **simpler and faster to use**.

The Rotten tomatoes dataset is a less large (17.1 Mo), but it also contains many ususeful features. It will be cleaned and merged to the CMU dataset to be **simpler to use**.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import string
import requests
import matplotlib.pyplot as plt

In [2]:
# Load the pre-processed CMU movies dataset
movies = pd.read_csv('Data/movies_preprocessed.csv')
movies.head(2)

,Unnamed: 0,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_countries,Movie_main_genre,Character_name,Character_in_title
0,0,975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,['United States of America'],thriller,"['Akooshay', 'Lieutenant Melanie Ballard', 'De...",False
1,1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000,NaN,95.0,['United States of America'],thriller,"['Police Officer', 'Reporter', 'FBI Profiler R...",False


In [3]:
# Load the IMdB ratings dataset
ratings_imdb = pd.read_table('Data/ratings_imdb.tsv', sep='\t')
ratings_imdb.head(2)

,tconst,averageRating,numVotes
0,tt0000001,5.7,2005
1,tt0000002,5.8,269


In [4]:
# Load the IMdB movies dataset
movies_imdb = pd.read_csv('Data/movies_imdb.tsv', sep = '\t')
movies_imdb.head(2)

/var/folders/wz/0j36zxw93hzb5_5xyy8nms340000gn/T/ipykernel_1306/2857110836.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_imdb = pd.read_csv('Data/movies_imdb.tsv', sep = '\t')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"


In [5]:
# Load the Rotten tomatoes dataset
movies_rotten = pd.read_csv('Data/movies_rottentomatoes.csv', sep = ',')
movies_rotten.head(2)

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19


In [6]:
# Several movies have the same title, some movies with the same title are even released the same year.
# That's why the datasets will be merged on a set of parameters : [Movie title, Release date, Runtime]
duplicates_name_date = movies[movies.duplicated(subset=['Movie_name', 'Movie_release_date'], keep=False)]
print(duplicates_name_date.shape)
duplicates_name_date.head(2)

(48, 11)


,Unnamed: 0,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_countries,Movie_main_genre,Character_name,Character_in_title
51,51,7460227,/m/0kv80y,The Bridge,2006,NaN,69.0,['United States of America'],drama,"['Scientologist #3', 'Scientology Leader', 'Sc...",False
231,231,4953051,/m/0cwvf3,The Bridge,2006,NaN,94.0,"['United States of America', 'United Kingdom']",documentary,NaN,NaN


In [7]:
duplicates_name_date_runtime = movies[ movies.duplicated(subset=['Movie_name', 'Movie_release_date', 'Movie_runtime'], keep=False)]
print(duplicates_name_date_runtime.shape)

(4, 11)


In [8]:
# Clean the IMdB movies dataset :
# Make a deep copy of the database
movies_imdb_filtered = movies_imdb.copy(deep=True)

# Remove unuseful columns
drop_col = ['titleType', 'primaryTitle', 'isAdult', 'genres', 'endYear']
movies_imdb_filtered.drop(drop_col, axis =1, inplace=True)

# Rename useful columns
movies_imdb_filtered = movies_imdb_filtered.rename(
    columns={
        "startYear": "Movie_release_date",
        "originalTitle": "Movie_name",
        "runtimeMinutes": "Movie_runtime",
    }
)

movies_imdb_filtered.head(2)

,tconst,Movie_name,Movie_release_date,Movie_runtime
0,tt0000001,Carmencita,1894,1
1,tt0000002,Le clown et ses chiens,1892,5


In [9]:
# Remove NaN values in the columns used for the merge
print('Size of the IMdB movies dataset with NaN values: ', movies_imdb_filtered.shape)

movies_imdb_filtered['Movie_release_date'] = movies_imdb_filtered['Movie_release_date'].replace('\\N', np.nan)
movies_imdb_filtered.dropna(subset=['Movie_release_date'], inplace =True)
pd.to_datetime(movies_imdb_filtered['Movie_release_date'], format='%Y') # keep only the year
movies_imdb_filtered['Movie_release_date'] = movies_imdb_filtered['Movie_release_date'].astype('int')

movies_imdb_filtered['Movie_runtime'] = movies_imdb_filtered['Movie_runtime'].replace('\\N', np.nan)
movies_imdb_filtered.dropna(subset=['Movie_runtime'], inplace =True)
movies_imdb_filtered['Movie_runtime'] = movies_imdb_filtered['Movie_runtime'].astype('float64', errors='ignore')

movies_imdb_filtered = movies_imdb_filtered.drop_duplicates( subset=["Movie_name", "Movie_release_date", "Movie_runtime"] )

print('Size of the IMdB movies dataset without NaN values: ', movies_imdb_filtered.shape)
movies_imdb_filtered.head(2)

Size of the IMdB movies dataset with NaN values:  (10293183, 4)
Size of the IMdB movies dataset without NaN values:  (2728882, 4)


,tconst,Movie_name,Movie_release_date,Movie_runtime
0,tt0000001,Carmencita,1894,1.0
1,tt0000002,Le clown et ses chiens,1892,5.0


In [10]:
# Remove the more recent movies
print('Size of the IMdB movies dataset with all release dates : ', movies_imdb_filtered.shape)
movies_imdb_filtered = movies_imdb_filtered[movies_imdb_filtered.Movie_release_date <= 2012]
print('Size of the IMdB movies dataset with : ', movies_imdb_filtered.shape)

Size of the IMdB movies dataset with all release dates :  (2728882, 4)
Size of the IMdB movies dataset with :  (1498895, 4)


In [11]:
# Clean the IMdB ratings dataset :
# Rename useful columns
ratings_imdb = ratings_imdb.rename(
    columns={
        "averageRating": "Audience_imdb_rating",
        "numVotes": "Audience_imdb_count"
    }
)

In [12]:
# Merge IMdB movies and ratings datasets
df_imdb = movies_imdb_filtered.merge(ratings_imdb, on='tconst', how='inner')
print('Size of the IMdB dataset with the ratings : ', df_imdb.shape)
df_imdb.drop('tconst', axis =1, inplace=True)
df_imdb.head(2)

Size of the IMdB dataset with the ratings :  (588499, 6)


,Movie_name,Movie_release_date,Movie_runtime,Audience_imdb_rating,Audience_imdb_count
0,Carmencita,1894,1.0,5.7,2005
1,Le clown et ses chiens,1892,5.0,5.8,269


In [13]:
# Clean the Rotten tomatoes dataset :
# Make a deep copy of the database
movies_rotten_filtered = movies_rotten.copy(deep=True)

# Remove unuseful columns
drop_col = ['rotten_tomatoes_link', 'critics_consensus', 'content_rating', 'genres', 'streaming_release_date', 'tomatometer_status', 'audience_status', 
            'tomatometer_top_critics_count', 'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count', 'movie_info']
movies_rotten_filtered.drop(drop_col, axis =1, inplace=True)

# Rename useful columns
movies_rotten_filtered = movies_rotten_filtered.rename(
    columns={
        "original_release_date": "Movie_release_date",
        "movie_title": "Movie_name",
        "runtime": "Movie_runtime",
        "directors": "Directors",
        "authors": "Authors",
        "actors":"Actors_rotten",
        "tomatometer_rating": "Critics_rotten_rating",
        "tomatometer_count": "Critics_rotten_count",
        "audience_rating": "Audience_rotten_rating",
        "audience_count": "Audience_rotten_count",
        "production_company": "Production_company",
    }
)

movies_rotten_filtered.head(2)

,Movie_name,Directors,Authors,Actors_rotten,Movie_release_date,Movie_runtime,Production_company,Critics_rotten_rating,Critics_rotten_count,Audience_rotten_rating,Audience_rotten_count
0,Percy Jackson & the Olympians: The Lightning T...,Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,119.0,20th Century Fox,49.0,149.0,53.0,254421.0
1,Please Give,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,90.0,Sony Pictures Classics,87.0,142.0,64.0,11574.0


In [14]:
# Remove NaN values in the columns used for the merge
print('Size of the Rotten tomatoes dataset with NaN values: ', movies_rotten_filtered.shape)

movies_rotten_filtered.dropna(subset=['Movie_release_date'], inplace =True)
movies_rotten_filtered['Movie_release_date'] = pd.to_datetime(movies_rotten_filtered['Movie_release_date'])
movies_rotten_filtered['Movie_release_date'] = movies_rotten_filtered['Movie_release_date'].dt.year
movies_rotten_filtered['Movie_release_date'] = movies_rotten_filtered['Movie_release_date'].astype('int')

movies_rotten_filtered.dropna(subset=['Movie_runtime'], inplace =True)
movies_rotten_filtered['Movie_runtime'] = movies_rotten_filtered['Movie_runtime'].astype('float64', errors='ignore')

print('Size of the Rotten tomatoes dataset without NaN values: ', movies_rotten_filtered.shape)
movies_rotten_filtered.head(2)

Size of the Rotten tomatoes dataset with NaN values:  (17712, 11)
Size of the Rotten tomatoes dataset without NaN values:  (16382, 11)


,Movie_name,Directors,Authors,Actors_rotten,Movie_release_date,Movie_runtime,Production_company,Critics_rotten_rating,Critics_rotten_count,Audience_rotten_rating,Audience_rotten_count
0,Percy Jackson & the Olympians: The Lightning T...,Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010,119.0,20th Century Fox,49.0,149.0,53.0,254421.0
1,Please Give,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010,90.0,Sony Pictures Classics,87.0,142.0,64.0,11574.0


In [20]:
# Merging IMdB dataset and pre-processed CMU movies dataset on [Movie title, Release date, Runtime]
cmu_imdb = movies.merge(df_imdb, on=['Movie_name', 'Movie_release_date', 'Movie_runtime'], how='inner')
print('Size of the CMU dataset : ', movies.shape)
print('Size of the IMdB dataset : ', df_imdb.shape)
print('Size of the CMU-IMdB dataset : ', cmu_imdb.shape)
cmu_imdb.head(1)

Size of the CMU dataset :  (26772, 11)
Size of the IMdB dataset :  (588499, 5)
Size of the CMU-IMdB dataset :  (13825, 13)


,Unnamed: 0,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_countries,Movie_main_genre,Character_name,Character_in_title,Audience_imdb_rating,Audience_imdb_count
0,0,975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,['United States of America'],thriller,"['Akooshay', 'Lieutenant Melanie Ballard', 'De...",False,4.9,56887


The CMU dataset size is **reduced by half** once merged to the IMdB movies-ratings dataset. 

If the movie distributions and cofounders are taken into account, we think is **still statistically relevant** to perform analyses on that merged dataset.

In [21]:
# Merging Rotten tomatoes dataset and CMU dataset on [Movie title, Release date, Runtime]
cmu_rotten = movies.merge(movies_rotten_filtered, on=['Movie_name', 'Movie_release_date', 'Movie_runtime'], how='inner')
print('Size of the CMU dataset : ', movies.shape)
print('Size of the Rotten tomatoes dataset : ', movies_rotten_filtered.shape)
print('Size of the CMU-Rotten tomatoes dataset : ', cmu_rotten.shape)
cmu_rotten.head(1)

Size of the CMU dataset :  (26772, 11)
Size of the Rotten tomatoes dataset :  (16382, 11)
Size of the CMU-Rotten tomatoes dataset :  (2549, 19)


,Unnamed: 0,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_countries,Movie_main_genre,Character_name,Character_in_title,Directors,Authors,Actors_rotten,Production_company,Critics_rotten_rating,Critics_rotten_count,Audience_rotten_rating,Audience_rotten_count
0,8,171005,/m/016ywb,Henry V,1989,10161099.0,137.0,['United Kingdom'],drama,"['Henry V', 'Catherine of Valois', 'Duke Thoma...",True,Kenneth Branagh,Kenneth Branagh,"Kenneth Branagh, Derek Jacobi, Brian Blessed, ...",MGM Home Entertainment,100.0,40.0,89.0,13465.0


The CMU-Rotten tomatoes dataset contains only  **9.5% of the pre-processed CMU dataset**. The pre-processed Rotten tomatoes dataset size is way smaller than the pre-processed IMdB dataset size (16'382 << 588'499), which can explain why we obtain a five times smaller dataset from the merge of the first in comparison with the merge of the second.

It can be used for analysis, but we will need to be **careful with the interpretations** we draw from it.

In [22]:
# Merging Rotten tomatoes dataset and CMU-IMdB dataset on [Movie title, Release date, Runtime]
movies_ratings = cmu_imdb.merge(movies_rotten_filtered, on=['Movie_name', 'Movie_release_date', 'Movie_runtime'], how='inner')
print('Size of the CMU dataset : ', movies.shape)
print('Size of the CMU-IMdB dataset : ', cmu_imdb.shape)
print('Size of the Rotten tomatoes dataset : ', movies_rotten_filtered.shape)
print('Size of the CMU-IMdB-Rotten tomatoes dataset : ', movies_ratings.shape)
movies_ratings.head(1)

Size of the CMU dataset :  (26772, 11)
Size of the CMU-IMdB dataset :  (13825, 13)
Size of the Rotten tomatoes dataset :  (16382, 11)
Size of the CMU-IMdB-Rotten tomatoes dataset :  (1813, 21)


,Unnamed: 0,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_countries,Movie_main_genre,Character_name,...,Audience_imdb_rating,Audience_imdb_count,Directors,Authors,Actors_rotten,Production_company,Critics_rotten_rating,Critics_rotten_count,Audience_rotten_rating,Audience_rotten_count
0,8,171005,/m/016ywb,Henry V,1989,10161099.0,137.0,['United Kingdom'],drama,"['Henry V', 'Catherine of Valois', 'Duke Thoma...",...,7.5,31189,Kenneth Branagh,Kenneth Branagh,"Kenneth Branagh, Derek Jacobi, Brian Blessed, ...",MGM Home Entertainment,100.0,40.0,89.0,13465.0


The CMU-IMdB-Rotten tomatoes (or ratings) dataset contains only 13.1% of CMU-IMdB dataset and **6.7% of the pre-processed CMU dataset**.

It can be used for analysis, but we will need to be **careful with the interpretations** we draw from it.

In [18]:
# Save the dataframes that include ratings as csv 
cmu_imdb.to_csv('Data/cmu_imdb.csv')
movies_ratings.to_csv('Data/movies_ratings.csv')
cmu_rotten.to_csv('Data/cmu_rotten.csv')